In [4]:
from gradio_client import Client

client = Client("https://mbazanlp-finetuned-nllb-general-en-kin.hf.space/")
result = client.predict(
				"Howdy!",	
				"eng_Latn",	
								
				"kin_Latn",	
							
				api_name="/predict"
)
print(result)

Loaded as API: https://mbazanlp-finetuned-nllb-general-en-kin.hf.space/ ✔


ReadTimeout: The read operation timed out

In [5]:
import re

def fileToSent(filename):
    sentences = []
    with open(filename, 'r') as file:
        line_count = 0
        for line in file:
            line_sentences = re.split(r'(?<=[.!?])\s+', line.strip())
            for sent in line_sentences:
                if len(sent) > 10 and len(sent.split()) > 4 and len(sent.split()) < 10 and sent[0].isupper():
                    # Remove non-alphanumeric characters
                    sent = re.sub(r'[^a-zA-Z0-9\s]', '', sent)
                    # Remove extra white spaces within the sentence
                    sent = re.sub(r'\s+', ' ', sent).strip()
                    # Remove leading numbers until the sentence begins with a letter
                    sent = re.sub(r'^\d+\s', '', sent)
                    sentences.append(sent)
    return sentences

In [7]:
mag_sentences = fileToSent('coca/text_mag.txt')
fic_sentences = fileToSent('coca/text_fic.txt')
news_sentences = fileToSent('coca/text_news.txt')
spok_sentences = fileToSent('coca/text_spok.txt')
acad_sentences = fileToSent('coca/text_acad.txt')
web_sentences = fileToSent('coca/text_web.txt')
tvm_sentences = fileToSent('coca/text_tvm.txt')
# concat all sentences
sentences = mag_sentences + fic_sentences + news_sentences + spok_sentences + acad_sentences + web_sentences + tvm_sentences

In [8]:
print(fic_sentences[:5])

['Of a heroin overdose in a cheap motel', 'He died before he got old', 'Paine died in 1976 of a drug overdose', 'You know dateless wonders', 'I always preferred the big monster flicks']


In [9]:
# EngToKin (EnglishText) -> KinText
def EngToKin(EnglishText, client):
    result = client.predict(
                    EnglishText,	
                    "eng_Latn",	
                    "kin_Latn",
                    api_name="/predict"
    )
    return result


In [10]:
from deep_translator import GoogleTranslator
def BatchEngToKin(batch):
    my_translator = GoogleTranslator(source='en', target='rw')
    result = my_translator.translate_batch(batch)
    return result

In [14]:
translated = BatchEngToKin(spok_sentences[6000:7000])


OSError: Could not find a suitable TLS CA certificate bundle, invalid path: /usr/local/lib/python3.11/site-packages/certifi/cacert.pem

In [15]:
print(translated)

['OBRIEN Birumvikana', 'Ariko ubanza uyu ni UYU MUNSI kuri NBC', 'Ibyo ntibisanzwe', 'Ariko rero, umunyamategeko ni ko bimeze', 'Ntekereza ko nari kuba mwiza', 'Reka tubivuge gutya', 'Ahubwo barangije bamubura', 'Ntushobora rero guhindura ibitekerezo byabo', 'Kandi Bill nibyiza nabonye', 'Nta kibuto cyariho', 'Bahuriye muri gereza', 'Byari urubanza rubabaje', 'Narebye hirya no hino murabizi', 'Ngiye gutaha', 'Namuhaye irindi shoti mubuzima', 'Shyira hasi ibisimba', 'Abapolisi ntibagenda hirya no hino bica abantu', 'Uramukura mumuhanda', 'Uzamushakira ubutaha', "Phillips yemerewe kurekurwa by'agateganyo mu 1999", 'Rita Braver avuga inkuru yacu', 'Wigeze utekereza ku Gisirikare', 'Oya nibyo', 'Turashaka abakiri bato', 'Turashaka abakiri bato', 'Wigeze utekereza ku Gisirikare', 'Kuki utabikora', 'Ufite imyaka ingahe', 'Ugomba kugenda', 'Inyundo murugo inyungu zo kwiga', 'Wigeze kubyumva', 'Ibisobanuro byawe byikubye kabiri', 'Ufite uburambe ku kazi', 'Wabonye ibihembo', 'Muri rusange rero

In [ ]:
import nltk
from nltk.translate import bleu
#calculateBLEU(hypothesis, reference) -> BLEUscore (float between 0 and 1)
def calculateBLEU(hypothesis, reference):
    BLEUscore = nltk.translate.bleu_score.sentence_bleu([reference], hypothesis)
    return BLEUscore

In [ ]:
def DictBLEU(Sentences, testCol, outputCol):
    for i in Sentences:
        BLEUscore = calculateBLEU(Sentences[i][1], Sentences[i][testCol])
        Sentences[i][outputCol] = BLEUscore
    return Sentences

In [ ]:
DictBLEU(Sentences, 2, 5)
DictBLEU(Sentences, 3, 6)
DictBLEU(Sentences, 4, 7)

In [13]:
# Write the updated data to a new CSV file
import csv


with open('trans.csv', 'a', newline='') as csv_file:
    csv_writer = csv.writer(csv_file)
    for i in range(len(translated)):
        csv_writer.writerow([translated[i], spok_sentences[i+3000]])
        